# Slicer Tutorial Brain MRI

This tutorial shows you how to use the functions from 'functions_for_slicer.py'. Before you begin, please ensure you have all the required modules downloaded from the requirements.txt file.

In [3]:
import sys
import os

# Add slicer_functions/ to the Python path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..', 'slicer_functions')))

from basic_functions import *

In [4]:
# Load in the DICOM directory for the brain resection patient into slicer
current_dir = os.path.dirname(os.path.abspath('__file__'))
brain_resection_dicom_path = os.path.join(current_dir, '..', 'example_patients', 'cervical_cancer')

slicer.mrmlScene.Clear(0) # Clear the scene
load_DICOM(brain_resection_dicom_path)

Found Referenced PET series
Found Referenced PET series
Found Referenced PET series
Found Referenced PET series
Found Radiopharmaceutical Code C-B1031
Found Radiopharmaceutical Code C-B1031
Found Radiopharmaceutical Code C-B1031
Found Radiopharmaceutical Code C-B1031
Found Referenced PET series
Found Referenced PET series
Found Referenced PET series
Found Referenced PET series
Found Radiopharmaceutical Code C-B1031


['vtkMRMLScalarVolumeNode1',
 'vtkMRMLScalarVolumeNode2',
 'vtkMRMLScalarVolumeNode3',
 'vtkMRMLScalarVolumeNode4',
 'vtkMRMLScalarVolumeNode5',
 'vtkMRMLScalarVolumeNode6',
 'vtkMRMLSegmentationNode1']

In [5]:
# get the volume nodes and set the PET cmap to heat
ct_node, pet_node = get_ct_and_pet_volume_nodes()

# Set the PET colormap to heat
petColor = slicer.mrmlScene.GetFirstNodeByName('PET-Heat')
pet_node.GetVolumeDisplayNode().SetAndObserveColorNodeID(petColor.GetID())
pet_node.GetVolumeDisplayNode().AutoWindowLevelOn()

#  set the CT colormap to abdomen window/level
slicer.modules.volumes.logic().ApplyVolumeDisplayPreset(ct_node.GetVolumeDisplayNode(), 'CT_ABDOMEN')


# Set the foreground and background of the slice viewer
slicer.util.setSliceViewerLayers(background=ct_node, foreground=pet_node, foregroundOpacity = 0.2)

In [6]:
# Get the segmentation node
segmentation_node = slicer.mrmlScene.GetFirstNodeByClass('vtkMRMLSegmentationNode')
segmentation = segmentation_node.GetSegmentation()

In [7]:
# Set the contours as BinaryLabelMap for manipulation
segmentation_node.CreateBinaryLabelmapRepresentation()


: 

In [ ]:
segmentation_node.SetSourceRepresentationToBinaryLabelmap()

In [ ]:
# Get the names of all the segments in the segmentation node
all_segment_names = [segmentation.GetNthSegment(i).GetName() for i in range(segmentation.GetNumberOfSegments())]

# Get the volume nodes as numpy arrays
ct_numpy_array = slicer.util.arrayFromVolume(ct_node)
pet_numpy_array = slicer.util.arrayFromVolume(pet_node)

# Make a dictionary of the segment names, and their numpy array sampled from the ct
ct_segment_dict = {}
for segment_name in all_segment_names:
    segment_id = segmentation.GetSegmentIdBySegmentName(segment_name)
    if not segment_id:
        ct_segment_dict[segment_name] = slicer.util.arrayFromSegmentBinaryLabelmap(segmentation_node, segment_id. ct_node).astype(int)

## Let's calculate the max and average houndsfeild values of the feamur and lungs:

In [ ]:
# get the array of the left and right femur from the dictionary
left_femur = ct_segment_dict['Left Femur']
right_femur = ct_segment_dict['Right Femur']

left_femur_ct = bitwise_and_from_array(left_femur, ct_numpy_array)
right_femur_ct = bitwise_and_from_array(right_femur, ct_numpy_array)

# get the max and average values of the left and right femur
print(f"Left femur max: {left_femur_ct.max()}, Left femur average: {left_femur_ct.mean()}")
print(f"Right femur max: {right_femur_ct.max()}, Right femur average: {right_femur_ct.mean()}")

In [ ]:
# combine all the 5 lung lobes into one segment array
lung_array = np.zeros_like(ct_numpy_array)

for segment_name in all_segment_names:
    if "lung" in segment_name.lower():
        segment_id = segmentation.GetSegmentIdBySegmentName(segment_name)
        lung_array = bitwise_or_from_array(lung_array, ct_segment_dict[segment_name])

# add the lung array to slicer
ct_segment_dict['Lung'] = lung_array
add_segmentation_array_to_node(ct_segment_dict['Lung'], segmentation_node, 'Lung', ct_node)

lung_ct = bitwise_and_from_array(lung_array, ct_numpy_array)

# Get the max and average values of the lung
print(f"Lung max: {lung_ct.max()}, Lung average: {lung_ct.mean()}")